In [19]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [2]:
background = None

accumulated_weight = 0.5

roi_top = 20
roi_bottom=300
roi_right=50
roi_left=350

In [3]:
def calc_accum_avg(frame , accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    
    cv2.accumulateWeighted(frame , background , accumulated_weight)

In [4]:
def segment(frame , threshold_min=25):
    
    diff = cv2.absdiff(background.astype("uint8"),frame)
    
    ret,thresholded = cv2.threshold(diff , threshold_min , 255 , cv2.THRESH_BINARY)
    
    image , contours , hierarchy = cv2.findContours(thresholded.copy() , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0 : 
        return None
    
    else:
        # ASSUMING THE LARGEST EXTERNAN CONTOUR IN ROI IS THE HAND
        hand_segment = max(contours , key=cv2.contourArea)
        
        return (thresholded , hand_segment)

In [5]:
def count_finger(thresholded , hand_segment):
    
    conv_hull = cv2.convexHull(hand_segment)
   
    # TOP , check documentation if you want to understand
    #top =    tuple(conv_hull[ conv_hull[:,:,1].argmin() ])[0]
    #bottom = tuple(conv_hull[ conv_hull[:,:,1].argmax() ])[0]
    #left =   tuple(conv_hull[ conv_hull[:,:,0].argmin() ])[0]
    #right =  tuple(conv_hull[ conv_hull[:,:,0].argmax() ])[0]
    top =    tuple(conv_hull[ conv_hull[:,:,1].argmin() ][0])
    bottom = tuple(conv_hull[ conv_hull[:,:,1].argmax() ][0])
    left =   tuple(conv_hull[ conv_hull[:,:,0].argmin() ][0])
    right =  tuple(conv_hull[ conv_hull[:,:,0].argmax() ][0]) 
    
    cX = ( left[0] + right[0] )//2
    cY = ( top[1] + bottom[1] )//2
    
    distance = pairwise.euclidean_distances( [[cX,cY]] , Y=[left,right,top,bottom])[0]
    
    max_distance = distance.max()
    
    radius = int(0.9*max_distance)
    circumference = (2*np.pi*radius)
    
    #circular_roi = np.zeros( thresholded[:2] , dtype="uint8")
    circular_roi = np.zeros( thresholded.shape , dtype="uint8")
    
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    
    circular_roi = cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    
    image,contours,hiearchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    count = 0
    
    for cnt in contours:
        
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        out_of_wrist = (cY*1.25) > (y+h)
        
        limit_points = ( circumference*0.25 > cnt.shape[0]   )
        
        if out_of_wrist and limit_points:
            count+=1
            
    return count

In [6]:
def count_fingers(thresholded , hand_segment):
    
    conv_hull = cv2.convexHull(hand_segment)
        
    #top = tuple( conv_hull[ conv_hull[:,:,1].argmin()[0] ] )

    count = 1
    
    return count

In [18]:
cam = cv2.VideoCapture(0)

num_frames = 0

while True:
    
    ret , frame = cam.read()
    
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom , roi_right:roi_left]
    
    gray = cv2.cvtColor(roi , cv2.COLOR_BGR2GRAY)
    
    gray = cv2.GaussianBlur(gray, (7,7) , 0)
    
    if num_frames < 60 :
        calc_accum_avg(gray,accumulated_weight)
        
        if num_frames <= 59:
            cv2.putText(frame_copy,"WAIT. GETTING BACKGROUND",(20,300),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow("Finger Count",frame_copy)
    else:
        
        hand = segment(gray,threshold_min=5)
        
        if hand is not None:
            
            thresholded , hand_segment = hand
            
            # DRAWS CONTOURS AROUND REAL HAND IN LIVE STREAM
            cv2.drawContours(frame_copy , [hand_segment+(roi_right,roi_top)] , -1 , (255,0,0) , 5)
            
            fingers = count_finger(thresholded , hand_segment)
            
            cv2.putText( frame_copy , str(fingers) , (70,50) , cv2.FONT_HERSHEY_SIMPLEX , 1 , (0,0,255) , 2 )
            
            cv2.imshow( "Thresholded" , thresholded )
            
    cv2.rectangle( frame_copy , (roi_left,roi_top) , (roi_right,roi_bottom) , (0,0,255) , 5 )
    
    num_frames += 1
    
    cv2.imshow("Finger Count" , frame_copy)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
cam.release()
cv2.destroyAllWindows()

In [20]:
np.array([1,1])

array([1, 1])

In [26]:
donem1_kredi =np.array([5,4,4,4,4])
donem1_not = np.array([4,4,4,4,4])

donem2_kredi =np.array( [5,4,4,3,3])
donem2_not = np.array([3.5,4,4,3,4])

donem3_kredi = np.array([4,4,3,3,3,2])
donem3_not = np.array([3,3,4,3.5,3,4])

donem4_kredi = np.array([])
donem4_not = np.array([])

donem5_kredi = np.array([4,3,3,3])
donem5_not = np.array([3,3,2.5,3.5])

donem6_kredi = np.array([4,4,3,2])
donem6_not =np.array( [3.5,4,3.5,2.5])

donem7_kredi = np.array([4,3,3,3,2])
donem7_not = np.array([3,4,3,3,3])

donem8_kredi = np.array([4,3,3,3])
donem8_not = np.array([4,4,3.5,4])



In [30]:
sum(donem1_kredi * donem1_not ) / sum(donem1_kredi)

4.0

In [37]:
A = (sum(donem1_kredi * donem1_not )+
    sum(donem2_kredi * donem2_not )+
    sum(donem3_kredi * donem3_not )+
    sum(donem5_kredi * donem5_not )+
    sum(donem6_kredi * donem6_not )+
    sum(donem7_kredi * donem7_not )+
    sum(donem8_kredi * donem8_not ))

B = (sum(donem1_kredi)+
    sum(donem2_kredi)+
    sum(donem3_kredi)+
    sum(donem5_kredi)+
    sum(donem6_kredi)+
    sum(donem7_kredi)+
    sum(donem8_kredi))

In [40]:
pre_son_donem = A/B
pre_son_donem

3.5486725663716814

In [59]:
feedback = 2.5
os = 3
comparc = 3
capstone = 4

donem10_kredi = np.array([3,3,4,3])
donem10_not = np.array([feedback,os,comparc,capstone])

In [60]:
tahmin = ( A + sum(donem10_kredi*donem10_not) ) / ( B+sum(donem10_kredi) )
tahmin

3.503968253968254